In [1]:
# conda activate org2html
import pandoc
from pandoc.types import *
from bs4 import BeautifulSoup 
import subprocess
import os 
shell_cmd = "pandoc -s temp.html -o temp.org"
html_attr = """#+SETUPFILE:css/emacs-org.setup
[  [[./Index.html][Home]] ]  $&ensp;$  | $&ensp;$ $&ensp;$ [text] $&ensp;$  $&ensp;$ | $&ensp;$ $&ensp;$ [link] $&ensp;$ $&ensp;$ | $&ensp;$ $&ensp;$ [title] $&ensp;$ $&ensp;$ | $&ensp;$ $&ensp;$ [icon] $&ensp;$ $&ensp;$ | $&ensp;$ $&ensp;$ [target]  $&ensp;$ $&ensp;$ | $&ensp;$ $&ensp;$ [expanded]  $&ensp;$ $&ensp;$
-----"""

In [2]:
def tag_input(text_input,start_tag,end_tag):
    return f"{start_tag}\n{text_input}\n{end_tag}"
# output_doc =tag_input("asdfas\nasfdasd\nadfasd",'#+BEGIN_SRC python',"#+END_SRC")    
# myprint(output_doc)
def myprint(data,output_file_name,show=True):
    # print(output_file_name)
    file1 = open(output_file_name, "a")  # append mode
    file1.write(data)
    file1.close()
    if not show:
        pass
        # print(data)
    # open file in write mode and append the with each line

def write_html2org(jupyter_html_path,output_file_name,show=False):
    if os.path.isfile(output_file_name):
        os.remove(output_file_name)    
    with open(jupyter_html_path) as file:
        html =file.read()
    soup = BeautifulSoup(html) # If this line causes an error, run 'pip install html5lib' or install html5lib 
    all_find = soup.find_all('div')
    
    # result
    for i in all_find:
        # print(i.attrs)
        if len(i.attrs) >0 :
            if (i.attrs['class'] == ['jp-Cell', 'jp-MarkdownCell', 'jp-Notebook-cell']):
                result = i.find('div',attrs={'class':"jp-RenderedHTMLCommon jp-RenderedMarkdown jp-MarkdownOutput"})
                html_doc = pandoc.read(result)
                html_doc = pandoc.write(html_doc, format="html")
                with open('temp.html','w') as f: f.write(html_doc.replace("¶",""))
                res = subprocess.run (shell_cmd,shell=True, capture_output=True)  
                if res.returncode:
                    print('Eroro is subprocess:',res.returncode)
                    break
                with open ('temp.org','r') as g: myprint(g.read(),output_file_name,show=False)
                
            if 'jp-CodeCell' in i.attrs['class']:
                result = i.find('div',attrs={'class':"highlight hl-ipython3"})
                
                source_code = result.get_text() 
                if source_code.strip():
                    org_src =tag_input(source_code.strip(),'#+BEGIN_SRC python',"#+END_SRC\n")    
                    myprint(org_src,output_file_name,show)
        
                result_block =i.find('div',attrs={'class':'jp-OutputArea-child'})
                if result_block:       
                    org_block = tag_input(result_block,'#+BEGIN_EXPORT html',"#+END_EXPORT\n")
                    myprint(org_block,output_file_name,show)
    # return lines

def clean_org_lines(lines):
    index =0
    while index < len(lines):
        # print(index, len(lines))    
        line = lines[index]
        if (':PROPERTIES:' in lines[index]) or ( ':CUSTOM_ID:'in lines[index]) or (':END:' in lines[index]):
            # print(index,line,lines[index+1],lines[index+2],)
            del lines[index]
            index = index # no update
        elif '[[' in line and '[]]' in line:
            before_link,after_link = line.split("[[")
            new_line = before_link + after_link.split("[]]")[1]
            lines[index]= new_line
            # print("Old Line Index: ",index, line.strip())
            # print("New Line Index: ",index, new_line.strip())
            index = index +1
        else:
            index = index+1
    return lines


# Convert single " .html "  to " .org "

In [3]:
# file_name = # input 
# jupyter_html_path = os.path.join("/home/karthik/mynotes/Python-Modules/pandas_html",file_name)


jupyter_html_path ="/home/karthik/mynotes/Python-Modules/Numpy_Complete.html"


In [5]:
output_file_name = os.path.basename(jupyter_html_path)
output_file_name = os.path.splitext(output_file_name)[0] + '_org.org'
print(output_file_name)

write_html2org(jupyter_html_path,output_file_name,False)

with open(output_file_name) as file:
    lines =file.readlines()
lines = clean_org_lines(lines)    

header = f"""#+TITLE: {output_file_name.replace("_org.org",'')}\n{html_attr}""" 
lines.insert(0,header)

with open(output_file_name,'w') as wf:
    wf.writelines(lines)

Numpy_Complete_org.org


# Convert all " *.html " in given dir to " *.org " files and one concatinated file 

In [ ]:
file_list = ["01_01_io.html", "01_02_display.html", "02_01_display.html", "02_02_view.html", "04_dfinfo.html", "05_DfSubset.html", "06_plot.html", "07_Categorical.html", "08_Group.html", "09_Delete.html", "10_01_Merge.html", "10_02_Sort.html", "11_01_Iterator.html", "11_02_Append(Concat).html", "11_03_Pivot.html", "12_pandas_stack.html", "12_TimeSeries.html", "13_crosstab.html", "13_Miscellaneous.html", "14_HandelingData-DataCleaning.html", "15_melt.html", "nump_nditer.html", "pandas_concat.html"]


In [9]:
# # # # # # # # # # # # # # # # # # # # 
#   Part-I : convert all html files  to org files 
# # # # # # # # # # # # # # # # # # # # 
for file_name in file_list:
    jupyter_html_path = os.path.join("/home/karthik/mynotes/Python-Modules/pandas_html",file_name)
    # "/home/karthik/mynotes/Python-Modules/Numpy.html"
    output_file_name = os.path.basename(jupyter_html_path)
    output_file_name = os.path.splitext(output_file_name)[0] + '_org.org'
    print(output_file_name)
    
    write_html2org(jupyter_html_path,output_file_name,False)
    
    with open(output_file_name) as file:
        lines =file.readlines()
    lines = clean_org_lines(lines)    
    with open(output_file_name,'w') as wf:
        wf.writelines(lines)
# # # # # # # # # # # # # # # # # # # # 
#   Part-II : Convert all html files to org files then From the generatd org files 
#             Concatinate all org files into single org file with header   
# # # # # # # # # # # # # # # # # # # # 

complete_file = 'pandas_full.org'
complete_lines =[]
for file_name in file_list:
    # file_name = file_list[0]
    jupyter_html_path = os.path.join("/home/karthik/mynotes/Python-Modules/pandas_html",file_name)
    # "/home/karthik/mynotes/Python-Modules/Numpy.html"
    output_file_name = os.path.basename(jupyter_html_path)
    output_file_name = os.path.splitext(output_file_name)[0] + '_org.org'
    print(output_file_name)
    complete_lines.append(f'* File: {output_file_name}\n')
    with open(output_file_name,'r') as rd:
        lines = rd.readlines()
    complete_lines.extend(lines)
    
header = f"""#+TITLE: {complete_file.replace("_org.org",'')}\n{html_attr}""" 
complete_lines.insert(0,header)

with open(complete_file,'w') as wf:
    wf.writelines(complete_lines)    

01_01_io_org.org
01_02_display_org.org
02_01_display_org.org
02_02_view_org.org
04_dfinfo_org.org
05_DfSubset_org.org
06_plot_org.org
07_Categorical_org.org
08_Group_org.org
09_Delete_org.org
10_01_Merge_org.org
10_02_Sort_org.org
11_01_Iterator_org.org
11_02_Append(Concat)_org.org
11_03_Pivot_org.org
12_pandas_stack_org.org
12_TimeSeries_org.org
13_crosstab_org.org
13_Miscellaneous_org.org
14_HandelingData-DataCleaning_org.org
15_melt_org.org
nump_nditer_org.org
pandas_concat_org.org


01_01_io_org.org
01_02_display_org.org
02_01_display_org.org
02_02_view_org.org
04_dfinfo_org.org
05_DfSubset_org.org
06_plot_org.org
07_Categorical_org.org
08_Group_org.org
09_Delete_org.org
10_01_Merge_org.org
10_02_Sort_org.org
11_01_Iterator_org.org
11_02_Append(Concat)_org.org
11_03_Pivot_org.org
12_pandas_stack_org.org
12_TimeSeries_org.org
13_crosstab_org.org
13_Miscellaneous_org.org
14_HandelingData-DataCleaning_org.org
15_melt_org.org
nump_nditer_org.org
pandas_concat_org.org


In [1]:
# complete_lines